In [1]:
# Data Manipulation Libraries
import pandas as pd
import numpy as np
import datetime
import os

import matplotlib.pyplot as plt
!pip install plotly
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
!pip install lightgbm
from lightgbm import LGBMRegressor
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [2]:
retail = pd.read_csv('Train.csv')
retail_test = pd.read_csv('Test.csv')
retail_data = retail.copy()
retail_test_data = retail_test.copy()

#Drop Duplicate rows
retail_data.drop_duplicates(subset=None, keep='first', inplace=True)
#Only dropped one outlier
retail_data.drop(retail_data.loc[retail_data['UnitPrice']>35000,:].index,inplace=True)

# #No missing values
# import missingno as msno
# msno.matrix(retail_test)
# print(retail_test.isna().sum())
#Seperate Categorical and Numerical Columns
cat_cols = retail_data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = retail_data.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

retail_data.head()

['InvoiceDate']
['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID', 'Country']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [3]:
def sum_of_rolling_mean():
    retail_data.groupby('StockCode')['UnitPrice'].rolling(3,center=True,min_periods=1).mean().reset_index(drop=True).sum()
    
def drop_irrelavant_columns(df):
    df.drop(columns=['InvoiceNo','Description'],inplace=True)
    
def sample_by_hour_set_index(df):
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
#     df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'].strftime('%Y-%m-%d %H:%M:%S'))
    df.sort_values(by='InvoiceDate',inplace=True)

def get_uniques(df_train,df_test):
    unique_SC_test_values = df_test['StockCode'].unique()
    unique_SC_train_values = df_train['StockCode'].unique()
    unique_unseen_SC_test_values = []
    unique_common_SC_values = []
    for i in unique_SC_test_values:
        if i in unique_SC_train_values:
            unique_common_SC_values.append(i)
        else:
            unique_unseen_SC_test_values.append(i)

    unique = {'SC_test':unique_SC_test_values,'SC_train':unique_SC_train_values,'SC_common_values':unique_common_SC_values,'unseen_SC':unique_unseen_SC_test_values}
    return unique

def perform_ops_for_both_train_and_test(data):
    drop_irrelavant_columns(data)
    sample_by_hour_set_index(data)
    

In [4]:
perform_ops_for_both_train_and_test(retail_data)
perform_ops_for_both_train_and_test(retail_test_data)
unique = get_uniques(retail_data,retail_test_data)

In [5]:
unique['SC_test']

array([2649, 1489, 1480, ..., 3667, 3654, 2275])

In [6]:
unique_country = np.sort(retail_test_data['Country'].unique())

In [7]:
retail_test_data['UnitPrice'] = 0

In [14]:
unique['SC_common_values'][1]

1489

In [36]:
# country 14, 13 has less values can use for test - 35 largest values
from fbprophet import Prophet
pro_df = retail_data.loc[retail_data['StockCode']==1489 & (retail_data['Country']==35),['InvoiceDate','UnitPrice']]
pro_df['InvoiceDate'] = pd.to_datetime(pro_df['InvoiceDate'])
pro_df.rename(columns={'InvoiceDate':'ds','UnitPrice':'y'},inplace=True)
model = Prophet()
# fit the model
model.fit(pro_df)
future = pro_df.drop(columns=['y'])
# use the model to make a forecast
forecast = model.predict(future)
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 16.


                   ds      yhat  yhat_lower  yhat_upper
0 2010-12-01 08:45:00  0.880740    0.814573    0.939818
1 2010-12-10 10:56:00  0.951576    0.888528    1.013317
2 2011-01-05 14:48:00  0.853427    0.790265    0.910005
3 2011-01-17 11:46:00  0.792443    0.733467    0.852501
4 2011-01-18 14:04:00  0.795384    0.727570    0.858225


In [38]:
oosp = retail_test_data.loc[retail_test_data['StockCode']==1489 & (retail_test_data['Country']==35),['InvoiceDate']]
oosp.rename(columns={'InvoiceDate':'ds'},inplace=True)

In [39]:
forecast = model.predict(oosp)
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

                   ds      yhat  yhat_lower  yhat_upper
0 2010-12-09 14:49:00  0.934882    0.869026    0.995521
1 2010-12-10 12:33:00  0.895259    0.835456    0.959327
2 2011-01-20 10:48:00  0.825299    0.762156    0.887070
3 2011-01-31 09:57:00  0.807909    0.744435    0.868081
4 2011-02-04 10:31:00  0.870506    0.811218    0.932617


In [344]:
# #Country code 35
cc=35
retail_data_cc = retail_data[retail_data['Country']==cc]
retail_test_data_cc = retail_test_data[retail_test_data['Country']==cc]
retail_test_data_cc['UnitPrice'] = 0

for usct in unique['SC_common_values']:
        stockuniq = retail_data_cc[retail_data_cc['StockCode']==usct]
        stockuniqt = retail_test_data_cc[retail_test_data_cc['StockCode']==usct]
        stockuniqt['second'] = 'sc'

        complete = stockuniq.append(stockuniqt)
        complete.sort_index(inplace=True)
#       complete['rollmean12'] = complete['UnitPrice'].rolling(12,center=True,min_periods=1).mean()
#       complete['rollmean9']  = complete['UnitPrice'].rolling( 9,center=True,min_periods=1).mean()
# #     complete['rollmean6']  = complete['UnitPrice'].rolling( 6,center=True,min_periods=1).mean()
        complete['rollmean3']  = complete['UnitPrice'].rolling( 3,center=True,min_periods=1).mean()
#       complete['ffill']   = complete['UnitPrice'].ffill()         # previous month 
#       complete['bfill']   = complete['UnitPrice'].bfill()         # next month
#       complete['interp']  = complete['UnitPrice'].interpolate()   # mean of prev/next
        complete.loc[complete['second']=='sc','rollmean3'] = complete.loc[complete['second']=='sc','rollmean3'].replace(0,np.nan).fillna(method='bfill')
        retail_test_data_cc.loc[retail_test_data_cc['StockCode']==usct,'UnitPrice'] = complete.loc[complete['second']=='sc','rollmean3'].values

retail_test_data_cc.fillna(method='bfill',inplace=True)

In [345]:
retail_test_data_cc[retail_test_data_cc['Country']==cc]

,StockCode,Quantity,InvoiceDate,CustomerID,Country,UnitPrice
99853,2649,6,2010-12-01 08:26:00,17850.0,35,1.250000
109155,1489,6,2010-12-01 08:28:00,17850.0,35,1.050000
23483,1480,3,2010-12-01 08:34:00,13047.0,35,1.983333
25010,1191,6,2010-12-01 08:34:00,13047.0,35,1.096667
75341,989,80,2010-12-01 09:00:00,13748.0,35,1.475000
...,...,...,...,...,...,...
28886,2967,25,2011-12-09 12:31:00,15804.0,35,0.140000
4599,1269,12,2011-12-09 12:31:00,15804.0,35,0.260000
79865,1328,12,2011-12-09 12:31:00,15804.0,35,0.416667
32091,2178,3,2011-12-09 12:31:00,15804.0,35,1.983333


In [346]:
#Country code not 35
retail_data_other = retail_data[retail_data['Country']!=35]
retail_test_data_other = retail_test_data[retail_test_data['Country']!=35]
retail_test_data_other['UnitPrice'] = 0

for usct in unique['SC_common_values']:
    stockuniq = retail_data_other[retail_data_other['StockCode']==usct]
    stockuniqt = retail_test_data_other[retail_test_data_other['StockCode']==usct]
    first = stockuniq[['UnitPrice','Country']]
    first_dict = first.groupby('Country')['UnitPrice'].mean().to_dict()
    stockuniqt['second'] = 'sc'
    stockuniqt['UnitPrice'] = stockuniqt['Country']
    stockuniqt['UnitPrice'].replace(first_dict,inplace=True)
    retail_test_data_other.loc[retail_test_data_other['StockCode']==usct] = stockuniqt
    
retail_data.loc[retail_data['Country']!=35] = retail_data_other 
retail_test_data.loc[retail_test_data['Country']!=35] = retail_test_data_other

In [347]:
#except country 35, for everything else take mean and assign value
complete_rtd = retail_test_data_cc.append(retail_test_data_other)
complete_rtd

,StockCode,Quantity,InvoiceDate,CustomerID,Country,UnitPrice
99853,2649,6,2010-12-01 08:26:00,17850.0,35,1.250000
109155,1489,6,2010-12-01 08:28:00,17850.0,35,1.050000
23483,1480,3,2010-12-01 08:34:00,13047.0,35,1.983333
25010,1191,6,2010-12-01 08:34:00,13047.0,35,1.096667
75341,989,80,2010-12-01 09:00:00,13748.0,35,1.475000
...,...,...,...,...,...,...
112314,1415,12,2011-12-09 12:50:00,12680.0,13,1.644872
46150,1470,12,2011-12-09 12:50:00,12680.0,13,0.850000
106449,1246,8,2011-12-09 12:50:00,12680.0,13,1.933333
38095,1580,4,2011-12-09 12:50:00,12680.0,13,3.750000


In [348]:
complete_rtd.sort_index(inplace=True)

In [349]:
zeros_unique = complete_rtd[complete_rtd['UnitPrice']==0]['StockCode'].unique()

In [350]:
unseen = unique['unseen_SC']
seen_in_train = unique['SC_common_values']

In [351]:
unseen_and_zero = []
seen_but_zero = []
for i in zeros_unique:
    if(i in unseen):
        unseen_and_zero.append(i)
    elif(i in seen_in_train):
        seen_but_zero.append(i)

In [352]:
len(unseen), len(zeros_unique), len(seen_but_zero), len(unseen_and_zero)

(65, 163, 98, 65)

In [353]:
#We need to handle notseen 59 stockcodes with mean of country 35 because its the max - didn't affect the score much
#We need to handle seen but zero!!! This shouldn't happen
unseen_rtd = retail_test_data.loc[retail_test_data['StockCode'].isin(unseen)]

In [354]:
unseen_rtd

,StockCode,Quantity,InvoiceDate,CustomerID,Country,UnitPrice
7567,2932,2,2010-12-03 15:45:00,12725.0,18,0.0
24558,3564,10,2010-12-05 11:10:00,14800.0,35,0.0
109806,825,1,2010-12-05 12:29:00,18156.0,35,0.0
82724,2739,1,2010-12-05 12:42:00,12748.0,35,0.0
39592,54,25,2010-12-06 12:06:00,16719.0,35,0.0
...,...,...,...,...,...,...
16192,2739,10,2011-11-28 14:12:00,17806.0,35,0.0
98871,3288,1,2011-11-30 17:37:00,15444.0,35,0.0
117349,107,2,2011-12-04 10:51:00,16438.0,35,0.0
19063,3671,12,2011-12-05 15:48:00,13790.0,35,0.0


In [355]:
f = 1478
f in unique['SC_common_values']

True

In [356]:
complete_rtd.loc[complete_rtd['StockCode'].isin(unseen) & (complete_rtd['UnitPrice']==0) ] = retail_data[retail_data['Country']==35]['UnitPrice'].mean()
complete_rtd.loc[complete_rtd['StockCode'].isin(unseen) & (complete_rtd['UnitPrice']==0) ]

#Doing this actually increased the score by 0.0004 - But evaluation is only done with 70% of the data so its tricky.
#Could check if are purchaced by customer id from earlier

,StockCode,Quantity,InvoiceDate,CustomerID,Country,UnitPrice


In [357]:
up = complete_rtd['UnitPrice'].apply(lambda x: round(x, 2))
UnitPrice = pd.DataFrame(up.astype('float').values,columns=['UnitPrice'])
UnitPrice.to_csv('kkall3m01.csv', index=False, index_label=None)

In [358]:
complete_rtd[complete_rtd['UnitPrice'].isna()]

,StockCode,Quantity,InvoiceDate,CustomerID,Country,UnitPrice
